In [110]:
from __future__ import print_function
%matplotlib inline
import cv2
import os
from skimage import io, transform
import numpy as np
import random

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import torch
from torchvision import transforms, datasets, utils as vutils
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

plt.rcParams['figure.figsize'] = [10, 5]

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
## Params 

# Image path
training_data_full = "data/training/full"
training_data_empty = "data/training/empty"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 16

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 1

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 0

epochs = 3


# Train Test and Validate partitions
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

In [ ]:
class WormClassDataset(Dataset):
    """ Full and emty are the paths to the data contianing
    the list of images. 0 = empty, 1 = full.
    Transform is the pytorch transformations to apply
    """
    def __init__(self, full_path, empty_path, transform=None):
        self.full_path = full_path
        self.full = os.listdir(full_path)
        self.empty_path = empty_path
        self.empty = os.listdir(empty_path)
        
        self.balace_data()
        self.data = self.full + self.empty
        self.remove_ds()
        
        self.transform = transform
        
    def balace_data(self):
        """Takes images form the two classes and curates the data so it's 50/50"""
        d = [self.full, self.empty]
        small = np.argmin([len(d[0]), len(d[1])])
        
        new_idxs = random.sample(range(0, len(d[not small])), len(d[small])) 
        new = [d[not small][i] for i in new_idxs]
        print(len(new))
                                       
        if not small:
            self.empty = new
        elif small:
            self.full = new
            
    def remove_ds(self):
        if '.DS_Store' in self.data:
            self.data.remove('.DS_Store'
            
    def __len__(self):
        data_count = len(self.empty) + len(self.full)
        return data_count

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        if idx < len(self.full):
            _class = 1
            img_name = os.path.join(self.full_path, self.data[idx])  
        else:
            _class = 0
            img_name = os.path.join(self.empty_path, self.data[idx])  

        image = io.imread(img_name)
        image = cv2.normalize(image, image, 0, 255, cv2.NORM_MINMAX)

        sample = image

        if self.transform:
            sample = self.transform(sample)

        return {'image': sample, 'class': _class}

test = WormClassDataset(training_data_full, training_data_empty)
b = test.__getitem__(0)
print(b['class'])

## Transform the data

In [ ]:
# Resize, convert to grayscale, convert to tensor, normalize, and rotate.
data_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((image_size, image_size)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
#         transforms.Normalize((0.5), (0.5))
    ])

worm_class_dataset = WormClassDataset(training_data_full, training_data_empty, transform=data_transform)


# Init gpu device if present.
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
# Show sample image from the dataset.

rand_saple = worm_class_dataset.__getitem__(np.random.randint(0, len(worm_class_dataset)))
plt.imshow(torch.reshape(rand_saple['image'], (64, 64, 1)))
print(rand_saple['class'])



## Create the actual dataloader
### Also will split data into train test and validate
___Will shuffle the data, and batch it___

In [115]:
train_count = round(len(worm_class_dataset) * train_ratio)
val_test_count = len(worm_class_dataset) - train_count
val_count = round(len(worm_class_dataset) * val_ratio)
test_count = round(len(worm_class_dataset) * test_ratio)
# Training and val + test dataset.

# The datasets needed for training

train_set, val_test_set = torch.utils.data.random_split(worm_class_dataset, [train_count, val_test_count])
val_set, test_set = torch.utils.data.random_split(val_test_set, [val_count, test_count])

# Training DATA-LOADER
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)

## Begin designing one of the model options
___Will try a few things:____

In [116]:
class WormClassifier(nn.Module):
    
    def __init__(self, dim):
        super(WormClassifier, self).__init__()
        self.conv1_1 = nn.Conv2d(1, 64, 5, 1, padding=2)
        self.conv1_2 = nn.Conv2d(64, 64, 5, 1, padding=2)
        
        self.conv2_1 = nn.Conv2d(64, 128, 5, 1, padding=2)
        self.conv2_2 = nn.Conv2d(128, 128, 5, 1, padding=2)
        
        self.conv3_1 = nn.Conv2d(128, 256, 5, 1, padding=2)
        self.conv3_2 = nn.Conv2d(256, 256, 5, 1, padding=2)
 
        self.conv4_1 = nn.Conv2d(256, 512, 5, 1, padding=2)
        self.conv4_2 = nn.Conv2d(512, 512, 5, 1, padding=2)
        
#         self.conv5_1 = nn.Conv2d(512, 512, 5, 1 padding=2)
#         self.conv5_2 = nn.Conv2d(512, 512, 5, 1 padding=2)
        
        self.fc1 = nn.Linear(512 * 4 * 4, 256 * 4 * 4)
        self.fc2 = nn.Linear(4096, 2048)
        self.fc3 = nn.Linear(2048, 1)

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = F.max_pool2d(x, 2)
            
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = F.max_pool2d(x, 2)
        
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.max_pool2d(x, 2)
        
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.max_pool2d(x, 2)
        
        x = x.view(-1, 8192)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        
        return x

    
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [117]:
# Define the model

model = WormClassifier(image_size)
# model.apply(weights_init)  # Random starting weights.
sample = train_set[3]["image"].unsqueeze(1)
model(sample).shape

print(model)

WormClassifier(
  (conv1_1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv1_2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2_1): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2_2): Conv2d(128, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3_1): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3_2): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4_1): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4_2): Conv2d(512, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=8192, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=1, bias=True)
)


Choose loss function and setup optimizer

In [118]:
loss_func = nn.BCELoss()

empty_labe = 0
full_label = 1

optimizer = optim.Adam(model.parameters(), lr=lr, betas=(beta1, 0.999))

In [119]:
# Training Loop

losses = []
iters = 0

for epoch in range(0, epochs):
    
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data["image"], data["class"]
        labels = labels.view(16, 1).to(torch.float32)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # Pass model, backprop, optimize.
        outputs = model(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Print Stats
        running_loss += loss.item()
        if i % 10 == 5:
            print(f'[{epoch + 1}, {i}] loss: {running_loss / 5}')
            runnning_loss = 0.0
            
        
        
        

[1, 5] loss: 0.8342215657234192
[1, 15] loss: 2.2211396217346193
[1, 25] loss: 3.608360195159912
[1, 35] loss: 4.993241560459137
[1, 45] loss: 6.375700414180756
[1, 55] loss: 7.754965317249298


KeyboardInterrupt: 

In [ ]:
print(2)

In [100]:
len(train_dataloader)

79